In [2]:
from appJar import gui
import paho.mqtt.client as mqtt
import logging
import json
import stmpy
import sqlite3 as sq
conn = sq.connect('Purr-fec.db')

In [2]:
##Initialize Database, only run once
c=conn.cursor()
c.execute('''CREATE TABLE cats
             (name TEXT PRIMARY KEY, rfid TEXT)''')
c.execute('''CREATE TABLE cordinates
             (cat TEXT, longditude REAL, latitude REAL, datetime TEXT)''')

In [13]:
MQTT_BROKER = 'mqtt.item.ntnu.no'
MQTT_PORT = 1883
MQTT_TOPIC_INPUT = 'ttm4115/team_1/server'
MQTT_COLLAR = 'ttm4115/team_1/collar'
MQTT_DOOR = 'ttm4115/team_1/door'

class CatStm:
    def conn_error(self):
        print('Updates from cat {} not received'.format(self.name))
        #TODO: Sends something to user?
        
    def cat_in(self):
        command = {"command": "cat_in", "name": self.name}
        self.server.publish_command(command,MQTT_COLLAR)
    
    def cat_outside(self):
        command = {"command": "cat_outside", "name": self.name}
        self.server.publish_command(command,MQTT_COLLAR)

    def cat_outside_fence(self):
        command = {"command": "cat_outside_fence", "name": self.name}
        self.server.publish_command(command,MQTT_COLLAR)
    
    def start_tracking(self):
        command = {"command": "start_tracking", "name": self.name}
        self.server.publish_command(command,MQTT_COLLAR)
    
    def __init__(self, name, server):
        self.name = name
        self.server = server
        t0 = {'source':'initial',
            'target':'cat_home',
            }
        #Home
        s_home = {'name':'cat_home',
            'entry':'start_timer("t",5000); cat_in'}
        
        t_btn_out = {'source':'cat_home',
            'target':'cat_outside',
            'trigger':'btn_outside'    
             }

        t_door_out = {'source':'cat_home',
            'target':'cat_outside',
            'trigger':'door_opened'
            }
        t_self_home = {'source':'cat_home',
            'target':'cat_home',
            'trigger':'t',
            'effect':'conn_error'}
        
        #Outside
        s_outside = {'name':'cat_outside',
            'entry':'start_timer("t",2000); cat_outside'}
        
        t_btn_in = {'source':'cat_outside',
            'target':'cat_home',
            'trigger':'btn_inside'    
            }
        
        t_door_in  = {'source':'cat_outside',
            'target':'cat_home',
            'trigger':'door_opened'
            }
        
        t_self_outside = {'source':'cat_outside',
            'target':'cat_outside',
            'trigger':'t'
            }
        
        t_start_tracking = {'source': 'cat_outside',
            'target':'cat_tracking',
            'trigger':'start_tracking'}
        
        #Outside fence
        s_outside_fence = {'name':'cat_outside_fence',
            'entry':'start_timer("t",100); cat_outside_fence'}
        
        t_self_outside_fence = {'source':'cat_outside_fence',
            'target':'cat_outside_fence',
            'effect':'conn_error'}
        
        #Tracking
        s_tracking = {'name':'cat_tracking',
                'entry':'start_timer("t",100);start_tracking'}
        
        t_self_tracking={'source':'cat_tracking',
            'target':'cat_tracking',
            'effect':'conn_error'}
        
        t_rec_data = {''}
        
        
        transitions=[t0,t_btn_out,t_btn_in,t_door_out,t_door_in,t_self_home,t_self_outside]
        states=[s_home]
        self.stm = stmpy.Machine(name=name,transitions=transitions,obj=self,states=states) 

class Server:
    def register_cat(self,name,rfid):
        # SEND to door
        c=conn.cursor()
        c.execute('INSERT INTO cats VALUES(?,?)',(name,rfid))
        cat_stm = CatStm(name,self) 
        self.stm_driver.add_machine(cat_stm.stm)
        conn.commit()
        
    def on_connect(self, client, userdata, flags, rc):
        print("Connected")
        
    def on_message(self, client, userdata, msg):
        try:
            payload = json.loads(msg.payload.decode("utf-8"))
        except Exception as err:
            print("invalid")
            return
        #TODO: Who is it from, what should i do

    
    def get_cat_names(self):
        c=conn.cursor()
        c.execute('SELECT name FROM cats')
        results = c.fetchall()
        cats = [ i[0] for i in results]
        return cats
        
    def publish_command(self,command,topic):
        payload = json.dumps(command)
        self.mqtt_client.publish(topic, payload=payload, qos=2)
    
    def __init__(self):
        self._logger = logging.getLogger(__name__)
        print('logging under name {}.'.format(__name__))
        self._logger.info('Starting Component')
        
        self.mqtt_client = mqtt.Client()
        self.mqtt_client.on_connect = self.on_connect
        self.mqtt_client.on_message = self.on_message
        self.mqtt_client.connect(MQTT_BROKER, MQTT_PORT)
        self.mqtt_client.subscribe(MQTT_TOPIC_INPUT)
        self.mqtt_client.loop_start()
        
        self.stm_driver = stmpy.Driver()
        self.stm_driver.start(keep_active=True)
        
        #Starts sessions for already registered cats, if the cat is not home it will take some updates to converge
        cats = self.get_cat_names()
        for name in cats:
            cat_stm=CatStm(name,self)
            self.stm_driver.add_machine(cat_stm.stm)
        
        
    
    
    
    


# This class is supposed to emulate the browser connected via http. In this case the GUI calls the 
# relevant server-functions directly   
class User:
    def __init__(self,server):
        self.mqtt_client = mqtt.Client()
        self.server=server
        self.server.user=self

    def create_gui(self):
        self.app = gui()
        
        def req_pos():
            pass
        
        def new_cat_btn():
            self.app.showSubWindow('new_cat')
            
        def mon_cat_btn():
            cats = self.server.get_cat_names() #HTTP
            self.app.changeOptionBox('my_cats',cats)
            self.app.showSubWindow('monitor_cat')
            
        def send_new_cat():
            name = self.app.getEntry('Name')
            rfid = self.app.getEntry('RFID number')
            self.server.register_cat(name, rfid) #HTTP
            self.app.hideSubWindow('new_cat')
        
        def request_position():
            server.get_latest_positon()
            
        #Main
        self.app.addButton('New Cat', new_cat_btn)
        self.app.addButton('Follow a Cat', mon_cat_btn)
        
        #New cat subwindow
        self.app.startSubWindow('new_cat')
        self.app.addLabelEntry("Name")
        self.app.addLabelEntry("RFID number")
        self.app.addButton('Register',send_new_cat)
        self.app.stopSubWindow()
        
        #Monitor cat subwindow
        self.app.startSubWindow('monitor_cat')
        self.app.addLabel('Monitor Cat')
        self.app.addOptionBox('my_cats', [])
        self.app.addButton('Request Position',req_pos)        
        self.app.stopSubWindow()       

        self.app.go()


my_server = Server()
my_user = User(my_server)
my_user.create_gui()

logging under name __main__.
Connected
Updates from cat 1234 not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Purr not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Queen not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat loolo not received
cat in
Command{"command": "cat_in", "name": "loolo"}
Updates from cat 1234 not received
cat in
Command['cat_in', '1234']
Updates from cat Purr not received
cat in
Command['cat_in', 'Purr']
Updates from cat 1234 not received
cat in
Command["cat_in", "1234"]
Updates from cat Purr not received
cat in
Command["cat_in", "Purr"]
Updates from cat Queen not received
cat in
Command['cat_in', 'Queen']
Updates from cat Queen not received
cat in
Command["cat_in", "Queen"]
Updates from cat 1234 not received
cat in
Command{"command": "cat_in", "name": "1234"}
Updates from cat Purr not received
cat in
Command{"command": "cat_in", "name": "Purr"}
Updates from cat Queen not received
cat in
Command{

Updates from cat 1234 not received
cat in
Updates from cat Purr not received
cat in
Updates from cat Queen not received
cat in
Updates from cat 1234 not received
Updates from cat DFDf not received
Updates from cat Purr not received
Updates from cat Queen not received
Updates from cat loolo not received
Updates from cat loolo23412334 not received
Updates from cat 1234 not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Purr not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Queen not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat loolo not received
cat in
Command{"command": "cat_in", "name": "loolo"}
Updates from cat 1234 not received
cat in
Command['cat_in', '1234']
Updates from cat Purr not received
cat in
Command['cat_in', 'Purr']
Updates from cat 1234 not received
cat in
Command["cat_in", "1234"]
Updates from cat Purr not received
cat in
Command["cat_in", "Purr"]
Updates from cat Queen not received
cat in
Command['cat_

Updates from cat 1234 not received
cat in
Updates from cat Purr not received
cat in
Updates from cat Queen not received
cat in
Updates from cat 1234 not received
Updates from cat DFDf not received
Updates from cat Purr not received
Updates from cat Queen not received
Updates from cat loolo not received
Updates from cat loolo23412334 not received
Updates from cat 1234 not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Purr not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Queen not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat loolo not received
cat in
Command{"command": "cat_in", "name": "loolo"}
Updates from cat 1234 not received
cat in
Command['cat_in', '1234']
Updates from cat Purr not received
cat in
Command['cat_in', 'Purr']
Updates from cat 1234 not received
cat in
Command["cat_in", "1234"]
Updates from cat Purr not received
cat in
Command["cat_in", "Purr"]
Updates from cat Queen not received
cat in
Command['cat_

Updates from cat 1234 not received
cat in
Updates from cat Purr not received
cat in
Updates from cat Queen not received
cat in
Updates from cat 1234 not received
Updates from cat DFDf not received
Updates from cat Purr not received
Updates from cat Queen not received
Updates from cat loolo not received
Updates from cat loolo23412334 not received
Updates from cat 1234 not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Purr not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Queen not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat loolo not received
cat in
Command{"command": "cat_in", "name": "loolo"}
Updates from cat 1234 not received
cat in
Command['cat_in', '1234']
Updates from cat Purr not received
cat in
Command['cat_in', 'Purr']
Updates from cat 1234 not received
cat in
Command["cat_in", "1234"]
Updates from cat Purr not received
cat in
Command["cat_in", "Purr"]
Updates from cat Queen not received
cat in
Command['cat_

Updates from cat 1234 not received
cat in
Updates from cat Purr not received
cat in
Updates from cat Queen not received
cat in
Updates from cat 1234 not received
Updates from cat DFDf not received
Updates from cat Purr not received
Updates from cat Queen not received
Updates from cat loolo not received
Updates from cat loolo23412334 not received
Updates from cat 1234 not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Purr not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Queen not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat loolo not received
cat in
Command{"command": "cat_in", "name": "loolo"}
Updates from cat 1234 not received
cat in
Command['cat_in', '1234']
Updates from cat Purr not received
cat in
Command['cat_in', 'Purr']
Updates from cat 1234 not received
cat in
Command["cat_in", "1234"]
Updates from cat Purr not received
cat in
Command["cat_in", "Purr"]
Updates from cat Queen not received
cat in
Command['cat_

...

Updates from cat 1234 not received
cat in
Updates from cat Purr not received
cat in
Updates from cat Queen not received
cat in
Updates from cat 1234 not received
Updates from cat DFDf not received
Updates from cat Purr not received
Updates from cat Queen not received
Updates from cat loolo not received
Updates from cat loolo23412334 not received
Updates from cat 1234 not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Purr not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Queen not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat loolo not received
cat in
Command{"command": "cat_in", "name": "loolo"}
Updates from cat 1234 not received
cat in
Command['cat_in', '1234']
Updates from cat Purr not received
cat in
Command['cat_in', 'Purr']
Updates from cat 1234 not received
cat in
Command["cat_in", "1234"]
Updates from cat Purr not received
cat in
Command["cat_in", "Purr"]
Updates from cat Queen not received
cat in
Command['cat_

Updates from cat 1234 not received
cat in
Updates from cat Purr not received
cat in
Updates from cat Queen not received
cat in
Updates from cat 1234 not received
Updates from cat DFDf not received
Updates from cat Purr not received
Updates from cat Queen not received
Updates from cat loolo not received
Updates from cat loolo23412334 not received
Updates from cat 1234 not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Purr not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Queen not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat loolo not received
cat in
Command{"command": "cat_in", "name": "loolo"}
Updates from cat 1234 not received
cat in
Command['cat_in', '1234']
Updates from cat Purr not received
cat in
Command['cat_in', 'Purr']
Updates from cat 1234 not received
cat in
Command["cat_in", "1234"]
Updates from cat Purr not received
cat in
Command["cat_in", "Purr"]
Updates from cat Queen not received
cat in
Command['cat_

Updates from cat 1234 not received
cat in
Updates from cat Purr not received
cat in
Updates from cat Queen not received
cat in
Updates from cat 1234 not received
Updates from cat DFDf not received
Updates from cat Purr not received
Updates from cat Queen not received
Updates from cat loolo not received
Updates from cat loolo23412334 not received
Updates from cat 1234 not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Purr not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Queen not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat loolo not received
cat in
Command{"command": "cat_in", "name": "loolo"}
Updates from cat 1234 not received
cat in
Command['cat_in', '1234']
Updates from cat Purr not received
cat in
Command['cat_in', 'Purr']
Updates from cat 1234 not received
cat in
Command["cat_in", "1234"]
Updates from cat Purr not received
cat in
Command["cat_in", "Purr"]
Updates from cat Queen not received
cat in
Command['cat_

Updates from cat 1234 not received
cat in
Updates from cat Purr not received
cat in
Updates from cat Queen not received
cat in
Updates from cat 1234 not received
Updates from cat DFDf not received
Updates from cat Purr not received
Updates from cat Queen not received
Updates from cat loolo not received
Updates from cat loolo23412334 not received
Updates from cat 1234 not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Purr not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Queen not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat loolo not received
cat in
Command{"command": "cat_in", "name": "loolo"}
Updates from cat 1234 not received
cat in
Command['cat_in', '1234']
Updates from cat Purr not received
cat in
Command['cat_in', 'Purr']
Updates from cat 1234 not received
cat in
Command["cat_in", "1234"]
Updates from cat Purr not received
cat in
Command["cat_in", "Purr"]
Updates from cat Queen not received
cat in
Command['cat_

Updates from cat 1234 not received
cat in
Updates from cat Purr not received
cat in
Updates from cat Queen not received
cat in
Updates from cat 1234 not received
Updates from cat DFDf not received
Updates from cat Purr not received
Updates from cat Queen not received
Updates from cat loolo not received
Updates from cat loolo23412334 not received
Updates from cat 1234 not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Purr not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat Queen not received
cat in
Command"{\"command\": \"new_timer}"
Updates from cat loolo not received
cat in
Command{"command": "cat_in", "name": "loolo"}
Updates from cat 1234 not received
cat in
Command['cat_in', '1234']
Updates from cat Purr not received
cat in
Command['cat_in', 'Purr']
Updates from cat 1234 not received
cat in
Command["cat_in", "1234"]
Updates from cat Purr not received
cat in
Command["cat_in", "Purr"]
Updates from cat Queen not received
cat in
Command['cat_